In [ ]:
def df2sqlite(dataframe, path_db=None, tbl_name="import"):

    '''The function df2sqlite converts a dataframe into a squlite database.
    
    Args:
       dataframe (panda.DataFrame): the dataframe to convert in a data base
       path_db (Path): full pathname of the database
       tbl_name (str): name of the table
    '''
    
    # Standard library imports
    import sqlite3
    
    # 3rd party imports
    import pandas as pd

    if path_db is None:  # Connetion to the database
        conn = sqlite3.connect(":memory:")
    else:
        conn = sqlite3.connect(path_db)
        
    # Creates a database and a table
    cur = conn.cursor()
    col_str = '"' + '","'.join(dataframe.columns) + '"'
    cur.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ({col_str})")
    dataframe.to_sql(tbl_name, conn, if_exists='replace', index = False)

    #cols_type = ",".join(["?"] * len(dataframe.columns))
    #data = [tuple(x) for x in dataframe.values]
    #cur.execute(f"DROP TABLE IF EXISTS {tbl_name}")
    #col_str = '"' + '","'.join(dataframe.columns) + '"'
    #cur.execute(f"CREATE TABLE {tbl_name} ({col_str})")
    #cur.executemany(f"insert into {tbl_name} values ({cols_type})", data)
    #conn.commit()
    cur.close()
    conn.close()

In [ ]:
# Standard librariy imports
from pathlib import Path
import sqlite3

# 3rd party imports
import pandas as pd

# Database construction
db_name = 'SAINTINES.db' # Database file name
    
db_path =   Path('c:\Temp') / Path(db_name) # Database file name

df = pd.read_excel(r'C:\Users\franc\PVcharacterization_files\Copie de Suivi des modules en enceinte.xlsx')
df.columns = [x .replace(' ','_') for x in df.columns]
tbl_name = 'essai'
df2sqlite(df, path_db=db_path, tbl_name=tbl_name)

# Query the database
conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute('''  
            SELECT PROJET,COUNT(*) how_many
            FROM essai
            WHERE (DATE_ENTREE >{})
            GROUP BY PROJET
          '''.format("'2020-10-29'"))

dict_query = {i:row for i, row in enumerate(cur.fetchall())}


dg = pd.DataFrame.from_dict(dict_query).T
#dg.columns = df.columns
    
cur.close()
conn.close() 
dg